# Importing and Exporting Data

Data can be imported into BigQuery from a CSV file stored within Cloud Storage, or it can be streamed directly into BigQuery from Python code.

Similarly, the results of a query can be exported to Cloud Storage as a set of shards, or they can be streamed directory into a file within Datalab. Note that for larger data sizes, it is recommended to choose the former.

In [1]:
import gcp
import gcp.bigquery as bq
import gcp.storage as storage
import pandas as pd
from StringIO import StringIO

# Importing Data

The first step to analyzing and querying your data is importing it. For purposes of the demo, we'll create a temporary table, in a temporary dataset within BigQuery, using a small data file within Cloud Storage.

## Importing Data from Cloud Storage

In [2]:
%%storage read gs://cloud-datalab-samples/cars.csv cars

In [3]:
print cars

Year,Make,Model,Description,Price
1997,Ford,E350,"ac, abs, moon",3000.00
1999,Chevy,"Venture Extended Edition","",4900.00
1999,Chevy,"Venture Extended Edition",Very Large,5000.00
1996,Jeep,Grand Cherokee,"MUST SELL! air, moon roof, loaded",4799.00



In [4]:
# Create the schema, using the convenience of basing it on example DataFrame
df = pd.read_csv(StringIO(cars))
schema = bq.Schema.from_dataframe(df)

# Create the dataset
bq.DataSet('sample').create()

# Create the table
sample_table = bq.Table('sample.cars').create(schema = schema, overwrite = True)

In [5]:
sample_table.load('gs://cloud-datalab-samples/cars.csv', mode='append',
                  source_format = 'csv', csv_skip_header_rows = 1)

Job job_dJXw8XXm3KCZ4WbBhIvCEev_7cI completed

In [6]:
%%sql
SELECT * FROM sample.cars

(rows: 4, time: 0.9s, 228B processed, job: job_DyI7LvRVowx-21tHcfsZEj2C-yI)

## Importing Data from a DataFrame

In [7]:
cars2 = storage.Item('cloud-datalab-samples', 'cars2.csv').read_from()
df2 = pd.read_csv(StringIO(cars2))
df2

,Year,Make,Model,Description,Price
0,2010,Honda,Civic,NaN,15000
1,2015,Tesla,Model S,NaN,64900


In [8]:
df2.fillna(value='', inplace=True)
df2

,Year,Make,Model,Description,Price
0,2010,Honda,Civic,,15000
1,2015,Tesla,Model S,,64900


In [9]:
sample_table.insert_data(df2)

(rows: 4, data-studio-team:sample.cars)

# Exporting Data

## Exporting Data to Cloud Storage

In [ ]:
project = gcp.Context.default().project_id
sample_bucket_name = project + '-datalab-samples'
sample_bucket_path = 'gs://' + sample_bucket_name
sample_bucket_object = sample_bucket_path + '/tmp/cars.csv'

print 'Bucket: ' + sample_bucket_name
print 'Object: ' + sample_bucket_object

In [11]:
sample_bucket = storage.Bucket(sample_bucket_name)
sample_bucket.create()

In [12]:
table = bq.Table('sample.cars')
table.extract(destination = sample_bucket_object)

Job job_vK-RKR7ARWJ9MWX6jQfgtLa7Al4 completed

In [13]:
%%storage list $sample_bucket_path

Name,Type,Size,Updated
tmp/cars.csv,application/octet-stream,230,2015-10-05 06:22:31.004000+00:00


In [14]:
%%storage read $sample_bucket_object data

In [15]:
print data

Year,Make,Model,Description,Price
1997,Ford,E350,"ac, abs, moon",3000
1999,Chevy,Venture Extended Edition,,4900
1999,Chevy,Venture Extended Edition,Very Large,5000
1996,Jeep,Grand Cherokee,"MUST SELL! air, moon roof, loaded",4799



## Exporting Data to a Local File

In [16]:
table.to_file('/tmp/cars.csv')

In [17]:
%%bash
ls -l /tmp/cars.csv

-rw-r--r-- 1 root root 299 Oct  5 06:22 /tmp/cars.csv


In [18]:
lines = None
with open('/tmp/cars.csv') as datafile:
  lines = datafile.readlines()
print ''.join(lines)

Year,Make,Model,Description,Price
1997,Ford,E350,"ac, abs, moon",3000.0
1999,Chevy,Venture Extended Edition,,4900.0
1999,Chevy,Venture Extended Edition,Very Large,5000.0
1996,Jeep,Grand Cherokee,"MUST SELL! air, moon roof, loaded",4799.0
2015,Tesla,Model S,,64900.0
2010,Honda,Civic,,15000.0



# Cleanup

In [ ]:
sample_bucket.item('tmp/cars.csv').delete()
sample_bucket.delete()
bq.DataSet('sample').delete(delete_contents = True)